In [1]:
# pip/conda installed
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rtree
import xarray as xr
from dask.distributed import Client

In [2]:
from utils.hls import HLSBand
from utils.hls import HLSCatalog
from utils.hls import HLSTileLookup
from utils.hls import fia_csv_to_data_catalog_input
from utils.hls import scene_to_urls

## Setup necessary utility functions/classes

In [3]:
lookup = HLSTileLookup()

Reading tile extents...
Read tile extents for 56686 tiles


In [4]:
point_catalog = HLSCatalog.from_point_pandas(df=fia_csv_to_data_catalog_input('./fia_10.csv'), bands=[HLSBand.NIR_NARROW, HLSBand.QA], tile_lookup=lookup)

In [5]:
point_catalog.xr_ds

<xarray.Dataset>
Dimensions:  (index: 443)
Coordinates:
  * index    (index) int64 2 2 2 2 2 2 2 2 2 2 2 2 2 ... 8 8 8 8 8 8 8 8 8 8 8 8
Data variables:
    INDEX    (index) int64 2 2 2 2 2 2 2 2 2 2 2 2 2 ... 8 8 8 8 8 8 8 8 8 8 8 8
    INVYR    (index) int64 2017 2017 2017 2017 2017 ... 2015 2015 2015 2015 2015
    STATECD  (index) int64 9 9 9 9 9 9 9 9 9 9 9 9 9 ... 9 9 9 9 9 9 9 9 9 9 9 9
    lat      (index) float64 41.49 41.49 41.49 41.49 ... 41.32 41.32 41.32 41.32
    lon      (index) float64 -73.5 -73.5 -73.5 -73.5 ... -73.32 -73.32 -73.32
    year     (index) object 2.017e+03 2.017e+03 ... 2.015e+03 2.015e+03
    tile     (index) object '18TXL' '18TXL' '18TXL' ... '18TXL' '18TXL' '18TXL'
    scene    (index) object 'L309/HLS.L30.T18TXL.2017003.v1.4' ... 'S309/HLS....
    sensor   (index) object 'L' 'L' 'L' 'L' 'L' 'L' ... 'S' 'S' 'S' 'S' 'S' 'S'
    dt       (index) datetime64[ns] 2017-01-03 2017-01-05 ... 2015-12-30
Attributes:
    bands:    [<HLSBand.NIR_NARROW: 5>, <HLSBand.QA: 11>]

In [6]:
%%time
bbox = [-124.98046874999999, 24.367113562651262, -66.70898437499999, 49.49667452747045]
years = [2019]
bands=[HLSBand.NIR_NARROW, HLSBand.QA]
# bbox_catalog = HLSCatalog.from_bbox(bbox, years, landsat_bands, sentinel_bands, lookup)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.3 µs


In [7]:
def create_multiband_dataset(row, bands, chunks):
    '''A function to load multiple bands into an xarray dataset '''
    
    # Each image is a dataset containing both band4 and band5
    datasets = []
    for band, url in zip(bands, scene_to_urls(row['scene'], row['sensor'], bands)):
        da = xr.open_rasterio(url, chunks=chunks)
        da = da.squeeze().drop(labels='band')
        ds = da.to_dataset(name=band)
        datasets.append(ds)
    return xr.merge(datasets)

def create_timeseries_multiband_dataset(df, bands, chunks):
    '''For a single HLS tile create a multi-date, multi-band xarray dataset'''
    datasets = []
    for i,row in df.iterrows():
        try:
            # print('loading...', row['dt'])
            ds = create_multiband_dataset(row, bands, chunks)
            datasets.append(ds)
        except Exception as e:
            print('ERROR loading, skipping acquistion!')
            print(e)
    DS = xr.concat(datasets, dim=pd.DatetimeIndex(df['dt'].tolist(), name='time'))
    print('Dataset size (Gb): ', DS.nbytes/1e9)
    return DS

In [8]:
def get_mask(qa_band):
    """Takes a data array HLS qa band and returns a mask of True where quality is good, False elsewhere
    Mask usage:
        ds.where(mask)
        
    Example:
        qa_mask = get_mask(dataset[HLSBand.QA])
        ds = dataset.drop_vars(HLSBand.QA)
        masked = ds.where(qa_mask)
    """
    def is_bad_quality(qa):
        cirrus = 0b1
        cloud = 0b10
        adjacent_cloud = 0b100
        cloud_shadow = 0b1000
        high_aerosol = 0b11000000

        return (qa & cirrus > 0) | (qa & cloud > 0) | (qa & adjacent_cloud > 0) | \
            (qa & cloud_shadow > 0) | (qa & high_aerosol == high_aerosol)
    return xr.where(is_bad_quality(qa_band), False, True)  # True where is_bad_quality is False, False where is_bad_quality is True

In [9]:
client = Client("tcp://127.0.0.1:39943")
client

Client Scheduler: tcp://127.0.0.1:39943 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.68 GB


In [10]:
x_chunk = 366*2
y_chunk = 366*2
chunks = {'band': 1, 'x': x_chunk, 'y': y_chunk}
grps = list(point_catalog.xr_ds.groupby('INDEX'))
jobs = []
for idx, ds in grps:
    df = ds.to_dataframe()
    jobs.append((idx, df))

In [11]:
training_ds = create_timeseries_multiband_dataset(jobs[0][1], point_catalog.xr_ds.attrs['bands'], chunks)

Dataset size (Gb):  6.791629112


In [12]:
qa_mask = None
if HLSBand.QA in training_ds.data_vars:
    qa_mask = get_mask(training_ds[HLSBand.QA])
    training_ds = training_ds.drop_vars(HLSBand.QA)

In [13]:
masked = training_ds.where(qa_mask)

In [15]:
# Rename vars from Enum to string for saving
masked = masked.rename({var: var.name for var in masked.data_vars})

In [16]:
median = masked.groupby('time.month').median()

In [17]:
median

<xarray.Dataset>
Dimensions:     (month: 12, x: 3660, y: 3660)
Coordinates:
  * y           (y) float64 4.6e+06 4.6e+06 4.6e+06 ... 4.49e+06 4.49e+06
  * x           (x) float64 6e+05 6e+05 6.001e+05 ... 7.098e+05 7.098e+05
  * month       (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    NIR_NARROW  (month, y, x) float64 dask.array<chunksize=(1, 732, 732), meta=np.ndarray>

In [18]:
%%time

median.to_zarr("monthly_median_2.zarr")

CPU times: user 4.5 s, sys: 89 ms, total: 4.59 s
Wall time: 3min 53s


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


CPU times: user 5.21 s, sys: 1.18 s, total: 6.39 s
Wall time: 2min 46s


<xarray.Dataset>
Dimensions:             (month: 12, x: 3660, y: 3660)
Coordinates:
  * y                   (y) float64 4.6e+06 4.6e+06 ... 4.49e+06 4.49e+06
  * x                   (x) float64 6e+05 6e+05 ... 7.098e+05 7.098e+05
  * month               (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    HLSBand.NIR_NARROW  (month, y, x) float64 nan nan nan ... -28.0 -18.0 -9.0

CPU times: user 5.21 s, sys: 1.18 s, total: 6.39 s
Wall time: 2min 46s


<xarray.Dataset>
Dimensions:             (month: 12, x: 3660, y: 3660)
Coordinates:
  * y                   (y) float64 4.6e+06 4.6e+06 ... 4.49e+06 4.49e+06
  * x                   (x) float64 6e+05 6e+05 ... 7.098e+05 7.098e+05
  * month               (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    HLSBand.NIR_NARROW  (month, y, x) float64 nan nan nan ... -28.0 -18.0 -9.0